In [1]:
from __future__ import division
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS, Rating
import time
import json
import sys
from math import sqrt

In [2]:
from pyspark.sql.types import *


In [3]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark import SparkContext
sc = SparkContext()


In [4]:
def mapDict(vals):
        return_dict = {}
        for val in vals:
            return_dict[val[0]] = val[1]
        return return_dict

def findAvg(vals):
        '''Find average value for each row'''
        count = 0
        tot = 0
        for item, rating in vals.items():
            tot += rating
            count += 1
        average = float(tot) / count
        
        for item, rating in vals.items():
            vals[item] = rating - average
        vals['row_avg'] = average
        return vals

In [5]:
train = sc.textFile('./Train-New.csv')
test = sc.textFile('./Test-New.csv')

In [6]:
train.take(5)

['User-ID,unique_isbn,Book-Rating',
 '273418,0671521438,7',
 '267444,0345391055,8',
 '182838,0929264576,10',
 '36554,0679450378,9']

In [7]:
train_header = train.first()
train_filtered = train.filter(lambda row: row != train_header)

In [8]:
train_header

'User-ID,unique_isbn,Book-Rating'

In [9]:
test_header = test.first()
test_filtered = test.filter(lambda row: row != test_header)

In [10]:
train_rdd = train_filtered.map(lambda x: x.split(',')).map(lambda line_split: (line_split[1], (line_split[0], float(line_split[2]))))

In [11]:
test_rdd = test_filtered.map(lambda x: x.split(',')).map(lambda line_split: (line_split[1], line_split[0], float(line_split[2])))


In [12]:
training_group = train_rdd.groupByKey()
training_group.take(5)

[('0671521438', <pyspark.resultiterable.ResultIterable at 0x1a1025fa940>),
 ('0345391055', <pyspark.resultiterable.ResultIterable at 0x1a1025fa9d0>),
 ('0679450378', <pyspark.resultiterable.ResultIterable at 0x1a1025faa30>),
 ('0394572122', <pyspark.resultiterable.ResultIterable at 0x1a1025faa90>),
 ('1550135201', <pyspark.resultiterable.ResultIterable at 0x1a1025faaf0>)]

In [13]:
training_group = train_rdd.groupByKey().mapValues(list)
training_group_dict = training_group.mapValues(mapDict)
training_group_dict_avg = training_group_dict.mapValues(findAvg)

In [15]:
training_group_dict_avg.take(10)

[('0671521438', {'273418': -0.5, '153662': 0.5, 'row_avg': 7.5}),
 ('0345391055',
  {'267444': 0.5625,
   '123544': -6.4375,
   '27204': 1.5625,
   '41841': -4.4375,
   '61798': 1.5625,
   '3965': 1.5625,
   '14698': -0.4375,
   '26339': 1.5625,
   '234597': 1.5625,
   '209373': 0.5625,
   '44588': -2.4375,
   '180942': -1.4375,
   '41455': -2.4375,
   '219294': 0.5625,
   '112109': 0.5625,
   '126736': -1.4375,
   '67010': -0.4375,
   '104739': -0.4375,
   '127389': -0.4375,
   '62532': -0.4375,
   '150498': 2.5625,
   '144619': 2.5625,
   '228681': 2.5625,
   '11676': 1.5625,
   '159059': -1.4375,
   '123494': 2.5625,
   '135380': 0.5625,
   '111133': 2.5625,
   '22912': 0.5625,
   '87964': -2.4375,
   '65663': 2.5625,
   '207259': 0.5625,
   '213126': -0.4375,
   '260796': 1.5625,
   '51165': 0.5625,
   '240906': -1.4375,
   '31556': -2.4375,
   '159506': -0.4375,
   '141444': -2.4375,
   '29539': 0.5625,
   '223501': -0.4375,
   '176966': -1.4375,
   '126810': 0.5625,
   '159033': 

In [17]:
training_group_dict_one = training_group_dict_avg.map(lambda x: (1, x))
training_one_reduce = training_group_dict_one.groupByKey().mapValues(list).map(lambda x: x[1])

In [18]:
training_one_reduce.collect()

[[('0671521438', {'273418': -0.5, '153662': 0.5, 'row_avg': 7.5}),
  ('0345391055',
   {'267444': 0.5625,
    '123544': -6.4375,
    '27204': 1.5625,
    '41841': -4.4375,
    '61798': 1.5625,
    '3965': 1.5625,
    '14698': -0.4375,
    '26339': 1.5625,
    '234597': 1.5625,
    '209373': 0.5625,
    '44588': -2.4375,
    '180942': -1.4375,
    '41455': -2.4375,
    '219294': 0.5625,
    '112109': 0.5625,
    '126736': -1.4375,
    '67010': -0.4375,
    '104739': -0.4375,
    '127389': -0.4375,
    '62532': -0.4375,
    '150498': 2.5625,
    '144619': 2.5625,
    '228681': 2.5625,
    '11676': 1.5625,
    '159059': -1.4375,
    '123494': 2.5625,
    '135380': 0.5625,
    '111133': 2.5625,
    '22912': 0.5625,
    '87964': -2.4375,
    '65663': 2.5625,
    '207259': 0.5625,
    '213126': -0.4375,
    '260796': 1.5625,
    '51165': 0.5625,
    '240906': -1.4375,
    '31556': -2.4375,
    '159506': -0.4375,
    '141444': -2.4375,
    '29539': 0.5625,
    '223501': -0.4375,
    '176966':

In [19]:
training_data_compile = training_one_reduce.collect()
training_data_compile = training_data_compile[0]
training_compile_dict = {}

In [20]:
for i in range(len(training_data_compile)):
        training_compile_dict[training_data_compile[i][0]] = training_data_compile[i][1]

In [21]:
item_to_user_pre = train_rdd.map(lambda x: (x[1][0], x[0])).groupByKey().mapValues(list)
item_to_user_compile = item_to_user_pre.collect()
item_to_user_dict = {}

In [22]:
for i in range(len(item_to_user_compile)):
        item_to_user_dict[item_to_user_compile[i][0]] = set(item_to_user_compile[i][1])

In [23]:
test_data = test_rdd.collect()
RMSE_tmp = 0
tmp_result = []
pearson_threshold = 0.3
random_pred = 0
upper_limit = 150
lower_limit = 15

In [24]:
for test in test_data:
        '''Get all the rows corresponding to cur user and item of test dataset'''
        cur_item, cur_user = test[0], test[1]
        filtered_train = {}
        if cur_item not in training_compile_dict or cur_user not in item_to_user_dict:
            '''If it's an unseen business id, assign some random prediction'''
            prediction = 5
            random_pred += 1
        else:
            '''We want to attach row corresponding to current item'''
            filtered_train[cur_item] = training_compile_dict[cur_item]
            cur_item_info = filtered_train[cur_item]
            '''Get a list of user_id who contains the current item'''
            row_set = item_to_user_dict[cur_user]
            for row in row_set:
                if len(training_compile_dict[row]) > lower_limit and len(training_compile_dict[row]) < upper_limit:
                    filtered_train[row] = training_compile_dict[row]
                
            '''Compute Pearson for each row and add to the final result if Pearson
            passes the threshold value'''
            predict_num = 0
            predict_den = 0
            for item, user_list in filtered_train.items():
                if item != cur_item:
                    num = 0
                    den1 = 0
                    den2 = 0
                    for user, rating in user_list.items():
                        if user in cur_item_info and user != cur_user and user != 'row_avg':
                            num += rating * cur_item_info[user]
                            den1 += rating**2
                            den2 += (cur_item_info[user])**2
                    denom = sqrt(den1) * sqrt(den2)
                    if num == 0 or denom == 0:
                        pearson = 0
                    else:
                        pearson = float(num) / denom
                    if pearson > pearson_threshold:
                        predict_num += (filtered_train[item][cur_user] + filtered_train[item]['row_avg']) * pearson
                        predict_den += abs(pearson)
            if predict_num == 0 or predict_den == 0:
                prediction = cur_item_info['row_avg']
            else:
                prediction = float(predict_num) / predict_den
                prediction = (prediction + cur_item_info['row_avg']) / 2.0 
        '''Save the results which consists of user_id, business_id, ground truth and predicted'''
        tmp_result.append(((test[1], test[0], test[2]), prediction))
        '''Compile results for final MSE computation'''
        RMSE_tmp += (test[2] - prediction)**2

In [25]:
RMSE = sqrt(RMSE_tmp / len(test_data))

In [26]:
print("RMSE for item-item model: ", RMSE)

RMSE for item-item model:  2.3952283538316923
